In [14]:
import spacy
import pandas as pd

In [15]:
nlp = spacy.load('en_core_web_sm', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [12]:
df_train = pd.read_csv('data/train.csv')
texts_train = []
for doc in nlp.pipe(df_train['text'], batch_size=50, n_process=-1):
	texts_train.append("".join([ent.text_with_ws for ent in doc if not ent.ent_type_]))
df_train['text'] = texts_train
df_train.to_csv('data/train_noents.csv', index=False)

df_test = pd.read_csv('data/test.csv')
texts_test = []
for doc in nlp.pipe(df_test['text'], batch_size=50, n_process=-1):
	texts_test.append("".join([ent.text_with_ws for ent in doc if not ent.ent_type_]))
df_test['text'] = texts_test
df_test.to_csv('data/test_noents.csv', index=False)

In [20]:
df = pd.read_csv(f'./data/all_the_news1.csv')
df = df[(df.publication == 'Breitbart')]
top10_authors = df[['author','id']].groupby('author').count().sort_values(by='id', ascending=False).head(10).reset_index() # this == what the paper proposed
df = pd.merge(df, top10_authors[['author']], on='author', how='right')
df = df.groupby(by='author').sample(500, random_state=42)
df.to_csv('data/all_the_news.csv', index=False)

In [58]:
def check_entity(ent, author):
	if ent.ent_type:
		return False
	for word in author.split():
		if word.lower() in ent.text.lower():
			return False
	return True

In [59]:
texts = []
for doc, author in zip(nlp.pipe(df['content'], batch_size=50, n_process=-1), df['author']):
	texts.append("".join([ent.text_with_ws for ent in doc if check_entity(ent, author)]))
df['content'] = texts
df.to_csv('data/all_the_news_noents.csv', index=False)

In [4]:
# df = pd.read_csv('data/all_the_news.csv')
# texts = []
# for doc in nlp.pipe(df['content'], batch_size=50, n_process=-1):
# 	texts.append("".join([ent.text_with_ws for ent in doc if not ent.ent_type_]))
# df['content'] = texts
# df.to_csv('data/all_the_news_noents.csv', index=False)